# Equivalence between MPG and Ternary Max Atom systems
## 1. Equivalence between Max Atom and Ternary Max Atom System
The equivalence is proven in detail on the **Max Atom** markdown file.

### 1.1 Reduction
The main idea is for each constraint:
$$
R:x\le \max Y+c
$$
Where: 
- $x$ is a variable 
- $Y$ is a subset of variables having **more than** $2$ elements.
- $c$ is a constant.

We select two **distinct** variables $y,z\in Y$. We add a new variable $w,$ and reduce the arity of $R$ as follow:
$$
R':x \le \max Y'+c
$$
With, $Y'=(Y\cup\{w\})\setminus\{y,z\}.$

We also couple $w$ with $y,z$ by adding the following constraint:
$$
w\le \max(y,z)
$$
Note that the new relation is **ternary**, so it does not need to be reduced.

### 1.2 Induction
The construction will be applied iteratively, until the arity of each relation $R$ is **less or equal to** $3.$

Now, after reducing all high-arity constraints, we must augment **binary** constraints as follow:
$$
R:x\le \max(y)+c \iff R:x \le \max(y,y)+c  
$$
With that the new system will be a ternary system.
### 1.3 Example
#### 1.3.1 Max Atom System

In [1]:
import mpg.csp.max_atom as ma
import mpg.csp.constraint_satisfaction as cs
VG=cs.VariableGenerator(name="X")
S=ma.MaxAtomSystem()
x=VG()
y=VG()
z=VG()
S.add_constraint(x,[x,y,z],-1)
S.add_constraint(y,[x,y,z],-1)
S.add_constraint(z,[x,y,z],-1)
S

System of 3 max constraints:
	X[1] <= max(X[1],X[2],X[3]) + -1
	X[2] <= max(X[1],X[2],X[3]) + -1
	X[3] <= max(X[1],X[2],X[3]) + -1

#### 1.3.2 Equivalent Ternary System

In [2]:
S_eq=S.equivalent_system
S_eq

System of 4 equations:
	P[1] <= max(X[3],X[2]) + 0
	X[1] <= max(X[1],P[1]) + -1
	X[2] <= max(X[1],P[1]) + -1
	X[3] <= max(X[1],P[1]) + -1

#### 1.3.3 Solution

In [5]:
S.solve(method="ACOCXX")

{X[3]: '-inf', X[1]: '-inf', X[2]: '-inf'}

## 2. Equivalence between Min Max problem and Max atom
### 2.2 Example
#### 2.2.1 Min Max System

In [5]:
import mpg.csp.max_atom as ma
import mpg.csp.constraint_satisfaction as cs
VG=cs.VariableGenerator(name="X")
S=ma.MinMaxSystem()
x=cs.Variable(id=0,name="X")
y=cs.Variable(id=1,name="Y")
z=cs.Variable(id=2,name="Z")
S.add_constraint("max",x,[x,y,z],[1,2,-1])
S.add_constraint("min",y,[x,y,z],[3,2,-1])
S.add_constraint("max",z,[x,y,z],[5,1,-1])
S

System of 3 min-max constraints:
	X <= max(X+1,Y+2,Z+-1)
	Y <= min(X+3,Y+2,Z+-1)
	Z <= max(X+5,Y+1,Z+-1)

#### 2.2.2 Equivalent Max Atom System

In [6]:
S.equivalent_system

System of 11 max constraints:
	S[1] <= max(X,X) + 1
	S[2] <= max(Y,Y) + 2
	S[3] <= max(Z,Z) + -1
	X <= max(S[1],S[2],S[3]) + 0
	Y <= max(X,X) + 3
	Y <= max(Y,Y) + 2
	Y <= max(Z,Z) + -1
	S[4] <= max(X,X) + 5
	S[5] <= max(Y,Y) + 1
	S[3] <= max(Z,Z) + -1
	Z <= max(S[4],S[5],S[3]) + 0

#### 2.2.3 Solving the System

In [7]:
assignment=S.solve(L=-20,R=20)
assignment

{X: 20, Y: 19, Z: 20}

#### 2.2.4 Verifying assignment

In [8]:
S.admissible(assignment)

True

## 3. Equivalence between an MPG and a bipartite MPG
### 3.1 $\text{Bi-MPG}\le \text{MPG}$
This is immediate.

### 3.2 $\text{MPG}\le \text{Bi-MPG}$
- Let $G=(V,E,L,s)$ be a mean-payoff game.
- Let $\mathcal{P}=\{0,1\}$ the set of players
- For a player $p\in\mathcal{P},$ we will denote by $\bar{p}$ the other player.
- Let $V'=V\times \mathcal{P}$

We will build the following equivalent mean-payoff game $G'(V',E',L')$ with:
$$
\begin{align*}
E'&=\{((u,s),(v,\bar{s})),\quad (u,v)\in E, s\in \mathcal{P}\} \\
\forall (u,v)\in E,\forall s\in\mathcal{P},\quad L'((u,s),(v,s))&=L(u,v)
\end{align*}
$$
### 3.3 Example

In [1]:
import mpg.visualisation.game as vgame
import mpg.games.mpg as mpg
G=mpg.MeanPayoffGraph()
G.add_edge(0,1,5)
G.add_edge(0,2,-5)
G.add_edge(2,1,3)
G.add_edge(2,0,-6)
G.add_edge(1,0,-6)
vgame.MPGVisualisation(G)

MPGVisualisation(layout=Layout(height='500px', width='100%'))

In [22]:
vgame.MPGVisualisation(G.as_bipartite())

MPGVisualisation(layout=Layout(height='500px', width='100%'))

## 4. Equivalence between Bipartite MPG and Min Max System
### 4.1 $\text{Bi-MPG} \le \text{Min-Max System}$
- Let $G=(V_1,V_2,E,L)$ a bipartite MPG.
- Let $\text{Adj}(v)$ be the adjacency list of a vertex $v\in V=V_1\cup V_2$
- For each vertex $v\in V_1$, we introduce a variable $X_v$
- For each vertex $v\in V_2$, we introduce a variable $Y_v$

Now we propose the following system:
$$
\begin{align*}
\forall u\in V_1, X_u &\le \max_{v \in \text{Adj}(u)}(Y_v+L(u,v))\\
\forall u\in V_2, Y_u &\le \min_{v \in \text{Adj}(u)}(X_v+L(u,v))
\end{align*}
$$

This system is an equivalent min max system. See the paper "Mean-Payoff Games and the Max-Atom Problem" for a detailing the equivalence.

### 4.2 $\text{Min-Max System}\le \text{Bi-MPG}$
### 4.3 Example

In [2]:
MMS=G.as_min_max_system()
MMS

System of 6 min-max constraints:
	V_0(0) <= max(V_1(1)+5,V_1(2)+-5)
	V_1(1) <= min(V_0(0)+-6)
	V_1(0) <= min(V_0(1)+5,V_0(2)+-5)
	V_0(1) <= max(V_1(0)+-6)
	V_1(2) <= min(V_0(1)+3,V_0(0)+-6)
	V_0(2) <= max(V_1(1)+3,V_1(0)+-6)

In [3]:
MMS.solve()

{V_1(0): -inf,
 V_1(2): -inf,
 V_0(0): -inf,
 V_1(1): -inf,
 V_0(2): -inf,
 V_0(1): -inf}

### 4.4 Optimal Strategy

In [4]:
S1,S2=mpg.optimal_strategy_pair(G)
S1,S2

({0: 2, 1: 0, 2: 0}, {0: 2, 1: 0, 2: 0})

### 4.5 Visualisation

In [5]:
A=vgame.MPGVisualisation(G)
A.set_node_color_mapping(vgame.WinnerVisualiser(A.graph,S1,S2))
A.set_edge_color_mapping(vgame.StrategyVisualiser(S1,S2))
A

MPGVisualisation(layout=Layout(height='500px', width='100%'))

### 4.6 Mean Pay Off

In [6]:
mpg.mean_payoffs(G,S1,S2)

{(0, False): -5.5,
 (2, True): -5.5,
 (1, 1): -5.5,
 (0, True): -5.5,
 (2, False): -5.5,
 (1, 0): -5.5}